In [ ]:
# General Libraries
%matplotlib inline
import pandas as pd
import numpy as np
import os               # para manipulacion de directorios
import shutil           # para eliminar carpetas no vacias
!pip install mglearn    # el comando 'import mglearn' no funcionaba
!pip install -U imbalanced-learn

from sklearn.feature_extraction.text import TfidfVectorizer

# Creating the modeling dataset
from sklearn.datasets import make_classification
# Data processing
import pandas as pd
import numpy as np
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Model and performance
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
# Oversampling and under sampling
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

# Augment

!pip install nlpaug
!pip install transformers
!pip install sacremoses
!pip install sentencepiece
!pip install translate
!pip install --upgrade translatepy
!pip install nltk


from translatepy import Translator
import time
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# cargamos el modelo pre-entrenado y el tokenizador
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')


In [ ]:
# Select the Personality

# 0: Agrado             # Score actual: 0.6830 (svc) | 0.6832 (knn)         | 0.6878(knn & COUNT) |
# 1: Apertura           # Score actual: 0.5916 (svc) | 0.5125 (knn)         | 0.6625 (dt & mf=400 & TF)    | 0.683 (dt & mf=70 & COUNT) | 0.6958 (dt & mf=60 & COUNT & cv=10) | 0.720 (dt & mf=175 & TFIDF & cv=15) | 0.754 (dt & mf=175 & COUNT & cv=15)
# 2: Extraversion       # Score actual: 0.6800 (svc) | 0.6880 (knn)         | 0.7000 (dt & COUNT) |
# 3: Neuroticismo       # Score actual: 0.5819 (svc) | 0.5887 (lr & COUNT)  | 0.6248 (dt & TF)    | 0.6999 (GridSearch Pipeline)
# 4: Responsabilidad    # Score actual: 0.7750 (svc) | 0.7833 (knn & TFIDF) | 0.7875 (dt & TF)    | 0.7958 (dt & TF & not_processed_texts) | 0.8041 (dt & TF & not_processed_texts & cv=10)

POS = 4

personality = ["Agrado", "Apertura", "Extraversion", "Neuroticismo", "Responsabilidad"]
colum_title = ["AgradPerc", "ApertPerc", "ExtravPerc", "NeurPerc", "RespPerc"]
path_colab  = [
    "https://drive.google.com/file/d/138orceXr83_CdJiVvC7bb9IF4-J8P6fp/view", #"./Agrado_Full.csv",
    "https://drive.google.com/file/d/1598rz27p2WRDtq6W1kQF1Pp0uigzpB_U/view", #"./Apertura_Full.csv" ,
    "https://drive.google.com/file/d/187RaPP8EnCjetblpr6YuHuCIXuASVDoa/view", #"./Extraversion_Full.csv" ,
    "https://drive.google.com/file/d/18h2J-jBaPXbhROjLnglukvYaF_Mv4BmN/view", #"./Neuroticismo_Full.csv" ,
    "https://drive.google.com/file/d/1wr20m6Cr7HL91rnbM3t5tfDr5GTZcHpS/view", #"./#Responsabilidad_Full.csv"

]


# Set values
path_colab = path_colab[POS]
colum_labels = colum_title[POS]

# With ONLY URL
if path_colab.startswith('http'):
  print("Ubication WEB")
  path_colab='https://drive.google.com/uc?id=' + path_colab.split('/')[-2]
else: print("Ubication Local")


Ubication WEB


In [ ]:
# Importar un CSV con pandas a un formato DataFrame (df)

df_ = pd.read_csv(
    path_colab,
    encoding = "utf-8",
    header = 0
)

df=df_


In [ ]:
# Sumar los valores de cada columna
sum1 = df['AgradPerc'].sum()
sum2 = df['ApertPerc'].sum()
sum3 = df['ExtravPerc'].sum()
sum4 = df['NeurPerc'].sum()
sum5 = df['RespPerc'].sum()

# Imprimir los resultados
print("Suma de AgradPerc:", sum1)
print("Suma de ApertPerc:", sum2)
print("Suma de ExtravPerc:", sum3)
print("Suma de NeurPerc:", sum4)
print("Suma de RespPerc:", sum5)



In [ ]:
texts_unprocessed = list(df["Autodescripción"]) #será el X_train o simplemente X es decir, los datos, en cross_val_score function
labels = list(df[colum_labels])                   #será el y_train o simplemente y es decir, los labels, en cross_val_score function que usará para evaluar el clasificador
print("Textos:   ", texts_unprocessed[0:3])
print("Etiquetas:", labels[0:3])

dataset_text_corpus_pos = df.loc[df['RespPerc'] == 1]
dataset_text_corpus_neg = df.loc[df['RespPerc'] == 0]
dataset_text_corpus = list(dataset_text_corpus_neg["Autodescripción"])
dataset_text_corpus_ids = list(dataset_text_corpus_neg["ID"])

print(len(dataset_text_corpus_neg))

Textos:    ['En general me considero una persona muy optimista; siempre le saco el lado positivo a las cosas. Me gusta mucho aprender y saber de todo un poco (por ejemplo; algún tema que me interese o auto de libro investigarlo a fondo) ; soy alegre y como cualidad tengo una escucha muy activa. Suelo salir de las situaciones incómodas con humor; busco bastante hacer reir a las demás personas ya que me hace bien a mi también. Me cuesta bastante establecer rutinas diarias y hacerlas perdurar en el tiempo; ya que tiendo a cambiar bastante de gustos y por lo general no me gusta hacer siempre lo mismo; por eso; busco establecer rutinas a corto plazo para poder cumplirlas y seguir motivada. Soy muy impaciente; cuando quiero algo lo quiero ya y eso a veces es contradictorio; aunque otras veces me motiva bastante para llegar a mis objetivos. Me cuesta establecer relaciones amorosas; supongo que por que busco relaciones donde prime la confianza no se busque sofocar al otro y viceversa. Tengo mu

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")



In [ ]:
from translatepy import Translator
import time
import nlpaug.augmenter.word as naw

def gptAugment(prompt, length):
    encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")   # tokenizamos el prompt
    output = model.generate(encoded_prompt, max_length=length*2, num_return_sequences=1, do_sample=True)    # generamos texto a partir del prompt
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)    # decodificamos el texto generado
    return generated_text

def backtranslate(text):

  time.sleep(1)  # Esperar 1 segundo antes de traducir nuevamente
  print("- back_translation_before=", text)

  translator = Translator()
  translation = (translator.translate(
      text=text,
      source_language='es',
      destination_language='en'
  )).result

  translation = translation.replace('.', '. ').replace(',', ', ').replace(';', '; ')
  count_words = len(translation.split())

  print("\n count: ",count_words)

  if (count_words > 500):
    count_words = 500

  translation = gptAugment(translation, count_words)

  translation = translation.split()[count_words:] # Quito las palabras que ingresaron
  translation = ' '.join(translation)

  time.sleep(1)  # Esperar 1 segundo antes de traducir nuevamente
  back_translation = translator.translate(
      translation,
      source_language='en',
      destination_language='es'
  ).result
  print("- back_translation_after=", back_translation)
  return back_translation


translation_list = []

# backtranslate(dataset_text_corpus[0])

print(dataset_text_corpus[0])

for text in dataset_text_corpus:
  print(f"\n translation_list: {len(translation_list)}")
  print(f"\n translation_list: {translation_list}")
  for ii in range(4):
    #print(f'Text to augment: {text}')

    # Normalizar texto
    half_length = len(text) // 2
    first_half = text[:half_length]
    second_half = text[half_length:]
    #print("first_half: ",first_half)
    #print("second_half: ",len(second_half))
    first_half = first_half.replace('.', '. ').replace(',', ', ').replace(';', '; ')
    second_half = second_half.replace('.', '. ').replace(',', ', ').replace(';', '; ')

    # first_half
    #print("\ncount: ",count_words1)
    translation1 = backtranslate(first_half)
    #print("T1: ",translation1)
    #translation1 = translation1.split()[count_words1:] # Quito las palabras que ingresaron
    #print("T1: ",translation1)
    #translation1 = ' '.join(translation1)
    #print("T3: ",translation1)
    translation_list.append(translation1)

    # second_half
    #count_words2 = len(second_half.split())
    #print("\ncount: ",count_words2)
    translation2 = backtranslate(second_half)
    #print("T1: ",translation2)
    #translation2 = translation2.split()[count_words2:] # Quito las palabras que ingresaron
    #print("T1: ",translation2)
    #translation2 = ' '.join(translation2)
    #print("T3: ",translation2)
    translation_list.append(translation2)

    #aux = backtranslate(text)
    #translation_list.append(aux)
    #if(len(translation_list) == 5):
    #  break




print(len(translation_list))


En general me considero una persona muy optimista; siempre le saco el lado positivo a las cosas. Me gusta mucho aprender y saber de todo un poco (por ejemplo; algún tema que me interese o auto de libro investigarlo a fondo) ; soy alegre y como cualidad tengo una escucha muy activa. Suelo salir de las situaciones incómodas con humor; busco bastante hacer reir a las demás personas ya que me hace bien a mi también. Me cuesta bastante establecer rutinas diarias y hacerlas perdurar en el tiempo; ya que tiendo a cambiar bastante de gustos y por lo general no me gusta hacer siempre lo mismo; por eso; busco establecer rutinas a corto plazo para poder cumplirlas y seguir motivada. Soy muy impaciente; cuando quiero algo lo quiero ya y eso a veces es contradictorio; aunque otras veces me motiva bastante para llegar a mis objetivos. Me cuesta establecer relaciones amorosas; supongo que por que busco relaciones donde prime la confianza no se busque sofocar al otro y viceversa. Tengo muchos amigos l

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


- back_translation_before= En general me considero una persona muy optimista;  siempre le saco el lado positivo a las cosas.  Me gusta mucho aprender y saber de todo un poco (por ejemplo;  algún tema que me interese o auto de libro investigarlo a fondo) ;  soy alegre y como cualidad tengo una escucha muy activa.  Suelo salir de las situaciones incómodas con humor;  busco bastante hacer reir a las demás personas ya que me hace bien a mi también.  Me cuesta bastante establecer rutinas diarias y hacerlas perdurar en el tiempo;  ya que tiendo a cambiar bastante de gustos y por lo general no me gusta hacer siempre lo mismo;  por eso;  busco establecer rutinas a corto plazo para poder cumplirlas y seguir motivada.  Soy muy impaciente;  cuando quiero algo lo quiero ya y eso a veces es contradictorio;  aunque otras veces me motiva bastante para llegar a mis objetivos.  Me cuesta establecer relaciones amorosas;  supongo que por que busco relaciones donde prime la confianza no se busque sofocar 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


- back_translation_before=  levanto;  desayuno;  me pongo a leer algo o estudiar;  luego limpio todo me baño y a veces sigo leyendo o me pongo a ver series hasta que me da hambre y como.  Pero a veces tengo que salir ya que cuido a mi sobrino mientras mi hermana trabaja y ahí aprovecho a hacer otras actividades con él;  por ejemplo jugar a la pelota a la play;  hacer cosas dulces para comer;  jugar a las cartas;  nos mantenemos bastante ocupados para no aburrirnos.  Cubro francos en un negocio;  esta semana solo me tocó trabajar una vez y por un lado estuvo bueno ya que hice algo distinto y me mantuve ocupada todo el día pero por el otro fué inquietante ver a la gente bastante paranoíca;  ver todo cerrado;  dejar entrar de a dos personas por vez;  etc.  Llegué a casa con esa sensación de estar contaminada con todo. . Como soy bastante de cambiar mis metas es dificil visualizarme en 5 años;  me gustaría ya haberme recibido;  ya que empecé la carrera tarde;  por que antes trabajé y me mu

...

- back_translation_before= Tiendo a ser una persona optimista;  intento ver lo positivo de los demás;  tiendo a caer en la imaginación y dejarme llevar fácilmente por esta;  soy bastante organizada respecto a mis cosas aunque por momento bastante perezosa (me cuesta empezar a trabajar en tareas que no me interesan;  como es el caso de las matemáticas).  Por otro lado;  soy graciosa (o eso intento);  me gusta hacer reír a los demás;  aunque a veces tiendo a ser egoísta y enfocarme en hacer lo que tengo que hacer.  En cuanto a aprender;  es una de las cosas que mas me gusta hacer;  ya sea aprender un idioma;  sobre una cultura;  sobre películas;  series;  sobre historia;  ciencia;  arte;  entre otras disciplinas.  También;  me gusta socializar con las personas;  aunque al principio soy tímida hasta poder entrar en confianza y cuando llego a este punto;  tiendo a hablar mucho y hacer muchos chistes.  Hace poco descubrí que me gusta mucho cocinar;  disfruto conocer nuevas recetas;  comidas

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


- back_translation_before= nte el ciclo lectivo (que se da entre marzo y junio;  luego agosto y diciembre) intentaba levantarme temprano;  dependiendo de las actividades que debía realizar ese día;  ya sea asistir a una clase teórica;  a una clase de trabajo practico;  estudiar;  pagar cuentas;  ir al supermercado;  entre otras actividades.  Antes de cada actividad tiendo a desayunar;  bañarme y disponerme a realizar la actividad que me corresponde;  e intentar terminarla lo antes posible;  para luego poder descansar o tener tiempo libre.  Y ahora;  durante el aislamiento obligatorio;  intento no dormir tanto;  llevar las tareas al día;  salir lo menos posible;  higienizarme de la manera correcta;  entre otras medidas. . Mi vida en cinco años;  espero que sea en un lugar donde pueda encontrarme siendo feliz;  ya sea con o sin un titulo;  rodeada de gente que me otorgue alegría (ya sea una pareja;  amigos;  familia;  mi mascota).  Mi vida ideal seria con el titulo de Licenciatura en Psi

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 count:  356


In [ ]:
print(translation_list[21])

¿En qué parte de Sudáfrica estoy actualmente?Estoy en la Universidad de Sudáfrica;Empecé a venir aquí en 1999. Entonces, en cierto sentido soy un alumbre;Sí;Y que también estoy involucrado en la filosofía y la música, y me veo en ambos campos;Siempre estoy involucrado en ambos.Y he estado aquí durante al menos 12 años.Comencé mi carrera;He estado empleado aquí;He tenido mi parte de entrevistas de trabajo;Hay una variedad de puestos, pero creo que como estudiante tengo una buena oportunidad;poder tomar cursos;Y me gustaría quedarme aquí;Durante mucho tiempo, y espero que jubilen trabajar durante mucho tiempo;Y que obtendría mi título;en la mejor escuela de psicología;Y que debería, dentro de la profesión de psicología profesional;Realmente tengo la opción de seguir psicología;Estoy muy interesado en cómo se aplica la filosofía;Y sigue y sigue y sigue;Realmente hay mucho que aprender ... Creo que es una gran escuela;Descubrí que me gusta venir aquí todos los meses.Pero en la actualidad, 

In [ ]:
index = 400

for textt in translation_list:
  nueva_fila = {'ID': index,'RespPerc': 1, 'Autodescripción': textt}
  df = df.append(nueva_fila, ignore_index=True)
  df = df.astype({'ID': 'int'})
  index +=1


<ipython-input-23-56b187dbcd60>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)
<ipython-input-23-56b187dbcd60>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)
<ipython-input-23-56b187dbcd60>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)
<ipython-input-23-56b187dbcd60>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)
<ipython-input-23-56b187dbcd60>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future vers

In [ ]:
print(df)
df.to_csv(r'Augment_gpt_RespPerc.csv', encoding = "utf-8", index = None, header=True)

      ID  RespPerc                                    Autodescripción
0      6         0  En general me considero una persona muy optimi...
1     10         0  En este momento de mi vida me siento una perso...
2     11         0  Tiendo a ser una persona optimista; intento ve...
3     13         0  Raro; perfil bajo en un ámbito social. En situ...
4     18         0  Me considero una persona agradable si se me tr...
..   ...       ...                                                ...
258  418         1  Come solos y los dos disfrutaríamos de una cen...
259  419         1  El corazón está en mis pulmones no solo en est...
260  420         1  puede ser reconocido.La diferencia entre la pr...
261  421         1  ¿En qué parte de Sudáfrica estoy actualmente?E...
262  422         1  Y así hace feliz a mi corazón ... Estoy muy ag...

[263 rows x 3 columns]


In [ ]:
index = 423

for textt in translation_list2:
  nueva_fila = {'ID': index,'RespPerc': 1, 'Autodescripción': textt}
  df = df.append(nueva_fila, ignore_index=True)
  df = df.astype({'ID': 'int'})
  index +=1

<ipython-input-27-74c47f85046d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)
<ipython-input-27-74c47f85046d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)
<ipython-input-27-74c47f85046d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)
<ipython-input-27-74c47f85046d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)
<ipython-input-27-74c47f85046d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future vers

In [ ]:
df['RespPerc'].value_counts()

print(df)
df.to_csv(r'Augment_gpt_RespPerc.csv', encoding = "utf-8", index = None, header=True)

      ID  RespPerc                                    Autodescripción
0      6         0  En general me considero una persona muy optimi...
1     10         0  En este momento de mi vida me siento una perso...
2     11         0  Tiendo a ser una persona optimista; intento ve...
3     13         0  Raro; perfil bajo en un ámbito social. En situ...
4     18         0  Me considero una persona agradable si se me tr...
..   ...       ...                                                ...
354  514         1   Fue extremadamente difícil para mí el año pas...
355  515         1   Los estudiantes de la familia de mi padre lle...
356  516         1   es algo que es una gran idea.Estoy buscando u...
357  517         1   puedo ver otro lado porque mi hermano me dejó...
358  518         1   El deseo para el mundo proviene de un objetiv...

[359 rows x 3 columns]


In [ ]:
from translatepy import Translator
import time
import nlpaug.augmenter.word as naw

def split_text(text, numer_of_splits):
  words = text.split()  # dividimos el texto por espacios en blanco
  num_words = len(words)
  chunk_size = num_words // numer_of_splits  # dividimos el número de palabras en 'numer_of_splits' partes iguales
  chunks = [words[i:i+chunk_size] for i in range(0, num_words, chunk_size)]  # creamos una lista con las partes iguales
  array_of_strings = [' '.join(array) for array in chunks]
  return array_of_strings

def gptAugment(prompt, length):
    encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")   # tokenizamos el prompt
    output = model.generate(encoded_prompt, max_length=length*2, num_return_sequences=1, do_sample=True)    # generamos texto a partir del prompt
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)    # decodificamos el texto generado
    return generated_text

def backtranslate(text):

  time.sleep(2)  # Esperar 1 segundo antes de traducir nuevamente
  print("\n- back_translation_before=", text)

  translator = Translator()
  translation = (translator.translate(
      text=text,
      source_language='es',
      destination_language='en'
  )).result

  translation = translation.replace('.', '. ').replace(',', ', ').replace(';', '; ')
  count_words = len(translation.split())

  #print("\n count: ",count_words)

  if (count_words > 500):
    count_words = 500

  translation = gptAugment(translation, count_words)

  translation = translation.split()[count_words:] # Quito las palabras que ingresaron
  translation = ' '.join(translation)

  time.sleep(2)  # Esperar 1 segundo antes de traducir nuevamente
  back_translation = translator.translate(
      translation,
      source_language='en',
      destination_language='es'
  ).result
  print("- back_translation_after=", back_translation)
  return back_translation


translation_list = []

# backtranslate(dataset_text_corpus[0])

print(dataset_text_corpus[0])

iterations = 0

augment_dataset = []

for text in dataset_text_corpus:
  print(f"\n translation_list: {len(augment_dataset)}")
  #print(f"Input text: {text}")
  # Genero 5 textos nuevos a partir del texto original
  text_generated = [] # Aqui alacenaremos los 5 textos nuevos
  for j in range(5):

    # Separo el texto en 4 partes para que el prompt no se sature (cada prompt acepta hasta 500 palabras aprox como maximo)
    splitted_text = split_text(text,4)

    text_concatenated = [] # Aqui vamos a ir concatenando los textos que fueron generados a partir de los 4 fragmentos

    for i in range(4):
      #print(f"fragment {i}: {splitted_text[i]}")
      generated = backtranslate(splitted_text[i])
      text_concatenated.append(f'. {generated}')

    text_generated.append(text_concatenated)

  #print(f"Output texts: {text_generated}")
  augment_dataset.append(
      {
        'ID': iterations,
        'match_id' : dataset_text_corpus_ids[iterations],
        'text_list': text_generated
      }
  )
  iterations+=1

  print(f"augment_dataset: {augment_dataset}")

    ## Normalizar texto
    #half_length = len(text) // 2
    #first_half = text[:half_length]
    #second_half = text[half_length:]
    ##print("first_half: ",first_half)
    ##print("second_half: ",len(second_half))
    #first_half = first_half.replace('.', '. ').replace(',', ', ').replace(';', '; ')
    #second_half = second_half.replace('.', '. ').replace(',', ', ').replace(';', '; ')

    ## first_half
    ##print("\ncount: ",count_words1)
    #translation1 = backtranslate(first_half)
    ##print("T1: ",translation1)
    ##translation1 = translation1.split()[count_words1:] # Quito las palabras que ingresaron
    ##print("T1: ",translation1)
    ##translation1 = ' '.join(translation1)
    ##print("T3: ",translation1)
    #translation_list.append(translation1)

    ## second_half
    ##count_words2 = len(second_half.split())
    ##print("\ncount: ",count_words2)
    #translation2 = backtranslate(second_half)
    ##print("T1: ",translation2)
    ##translation2 = translation2.split()[count_words2:] # Quito las palabras que ingresaron
    ##print("T1: ",translation2)
    ##translation2 = ' '.join(translation2)
    ##print("T3: ",translation2)
    #translation_list.append(translation2)

#print(len(translation_list))

me considero con una personalidad fuerte; normalmente me muestro bastante "mala" con la gente que no conozco; pero con mis amigos soy alguien distinto. soy medio distante; no soy de ser muy cariñosa; siempre depende la otra persona. siempre me considere muy divertida; me encanta hacer reír a los demás. soy muy deportista; me gusta estar en movimiento; desde los 4 años que hago deporte; hice danza y también salgo a trotar o andar en rollers. mis metas en la vida son recibirme; viajar y tener 2 hijos. hoy en día me siento bien; años anteriores no la estaba pasando muy bien; pero empece a ver el lado lindo de la vida; gracias a mis amigos y mi familia. aunque soy muy desapegada; la familia y amigos son algo muy importante en mi vida; actualmente vivo con mi tía (que es como mi mamá); ella es la persona más importante y después está papá; gracias a ella estoy donde estoy y estudiando lo que me gusta. siento que el por qué me gusta la psicología forma parte de mi personalidad; siempre me gu

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


- back_translation_after= Y no digo que no lo creo y lo amo;De hecho, me encanta aprender cosas nuevas;Mi novia era muy abierta y amigable y muy interesada en mí, y siempre supe, gracias a ella.Siempre trato de conectarme con personas que son positivas hacia mí;Hoy si no estoy solo, estoy con alguien y me siento bien conmigo mismo y pienso en mí mismo;Trato de tener una actitud positiva de la vida. Hace todo lo posible para obtener lo que quería a través de mi trabajo;Tengo amigos que espero conocer;Gracias a eso, mi vida ahora es estable e interesante.Es una vida divertida y divertida;Me encanta recibir cumplidos;Si es encontrar algo hermoso de lo que lo hice para ese propósito.

- back_translation_before= ayudar a los demás; en lo que pueda. me gusta tener la razón siempre jaja; así que soy de estar discutiendo y peleando todo el tiempo; siempre intento convencer a los demás de que si tengo razón. siento que esto es algo malo y algo bueno; malo porque me lleva a tener peleas y soy ba

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


- back_translation_after= Mi sobrina trabaja;Y un hotel.Y si es agradable y cálido y todos están felices de que se quede allí durante meses;Y estoy acostumbrado a vivir en una ciudad con un gran parque.Y no tengo miedo y voy a algún lugar donde no esté demasiado lleno.Así que hay algo de paz aquí y tengo muchos amigos porque paso tiempo con amigos en la ciudad aquí;Entonces, cuando nuestro tío y tía murieron, no tuve mucho contacto con ellos;Pero luego, cuando perdimos a mi sobrina, nos acercamos.Y vi las fotos que me envió.Y sé que estas fotos son las de mi tío;Y los encontré en su teléfono celular de su hija;Es curioso cómo esta hermosa niña vive en un Villageulia con su tío o tía.También quería algo de paz.Entonces

- back_translation_before= mi casa (un poco triste; porque extraño a matilda) pero nos vemos todos los días igual. lo malo de toda la semana es no poder salir a ningún lado; aunque en mi vida normal no salgo tanto. esta última semana conocí a una persona que se ganó mi c

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



- back_translation_before= que necesito que la gente me sea muy clara; que me demuestre todo el tiempo para no desconfiar. Pero a la vez yo tampoco digo que es eso lo que quiero y tampoco digo como me siento; estoy tratando de trabajar en eso. En dejarme a mi misma en claro que es lo que quiero y como me siento; para también poder transmitirlo a los demás y que haya mejor comunicación. Mi día a día; lo que estoy haciendo es levantarme; comer algo; ver una serie o película; limpiar; prender sahumerios o palo santo; estudiar un poco; jugar con mi gato; ver algún documental; alguna charla o algún stand up; hacer video llamads con mi familia; también con amigues. Hago una clase de funcional y una de yoga por día; medito las veces que pueda durante el dia; a la noche por lo general me mimo y me hago alguna comida rica; escucho casi todo el día música; Leo algunos poemas; me cuido superficialmente ( mascarilla; tratamiento del pelo; las uñas; etc). También juego a un jueguito. Y me duermo v

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



- back_translation_before= También me veo planeando los viajes que voy a hacer; cumpliendo mi sueño de poder conocer lugares; pero de mochilera; sin ninguna de las comodidades que tengo día a dia. En 5 años me veo bien; con una estabilidad emocional y con situaciones ya superadas. En 5 años me veo con todos los obstáculos que hoy me parecen imposibles; ya superados. Veo una versión de mi más sana; mas responsable; más estable; con relaciones sexoafectivas mas sanas. Me veo con mis amistades de siempre; apoyándonos en todo y alegrandonos por el logro de todas. A mi mamá y a mi papá más orgullosos de lo que están hoy en día. Felices por haberme recibido; el cual es su sueño y por lo que tanto se esforzaron para darme la mejor educación y todas las posibilidades de ser lo que yo quisiera. En 5 años me veo creando nuevos proyectos. Me gustaría también poder brindar ayuda con mi profesión a personas con bajos recursos económicos; en situaciones de pobreza. En 5 años me veo mucho más metida

NoResult: ignored

In [ ]:
index = 500

for textt in output_test_array:
  match_id = textt['match_id']
  print(match_id)

  # testeo si el ID existe en el Documento:
  if(len(df.loc[df['ID'] == match_id, 'AgradPerc'].values) == 0):
    print(f'No existe el ID: {match_id}')

  else:
    nueva_fila = {
      'ID': index,
      'AgradPerc':  df.loc[df['ID'] == match_id, 'AgradPerc'].values[0],
      'ApertPerc':  df.loc[df['ID'] == match_id, 'ApertPerc'].values[0],
      'ExtravPerc': df.loc[df['ID'] == match_id, 'ExtravPerc'].values[0],
      'NeurPerc':   df.loc[df['ID'] == match_id, 'NeurPerc'].values[0],
      'RespPerc':   df.loc[df['ID'] == match_id, 'RespPerc'].values[0],
      'Autodescripción': textt['text']
    }
    df = df.append(nueva_fila, ignore_index=True)
    df = df.astype({'ID': 'int'})
    index +=1

15
15
15
15
15
35
35
35
35
35


54
54
54
54
54
123
123
123
123
123
135


135
135
135
135
144
144
144
144
144


177
177
177
177
177
217
217
217
217
217


226
226
226
226
226
235
235
235


235
235


<ipython-input-13-912a7692c13e>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


In [ ]:
print(df)
df.to_csv(r'Augment_gpt4_RespPerc.csv', encoding = "utf-8", index = None, header=True)

      ID  AgradPerc  ApertPerc  ExtravPerc  NeurPerc  RespPerc  \
0     15          0          1           1         1         0   
1     35          0          0           0         0         0   
2     54          0          1           0         1         1   
3    123          0          0           0         0         0   
4    135          0          0           0         1         0   
..   ...        ...        ...         ...       ...       ...   
322  545          0          0           0         0         0   
323  546          0          0           0         0         0   
324  547          0          0           0         0         0   
325  548          0          0           0         0         0   
326  549          0          0           0         0         0   

                                       Autodescripción  
0    me considero con una personalidad fuerte; norm...  
1    Soy una persona por lo generalmente muy alegre...  
2    Soy una persona confiable; que 

In [ ]:
print(df)

      ID  AgradPerc  ApertPerc  ExtravPerc  NeurPerc  RespPerc  \
0     15          0          1           1         1         0   
1     35          0          0           0         0         0   
2     54          0          1           0         1         1   
3    123          0          0           0         0         0   
4    135          0          0           0         1         0   
..   ...        ...        ...         ...       ...       ...   
322  545          0          0           0         0         0   
323  546          0          0           0         0         0   
324  547          0          0           0         0         0   
325  548          0          0           0         0         0   
326  549          0          0           0         0         0   

                                       Autodescripción  
0    me considero con una personalidad fuerte; norm...  
1    Soy una persona por lo generalmente muy alegre...  
2    Soy una persona confiable; que 